# TP - Data Matching with Python Record Linkage Toolkit


During this course, we will use the built-in *Freely Extensible Biomedical Record Linkage 2 (Febrl2)* dataset in the *Python Record Linkage toolkit* (https://recordlinkage.readthedocs.io/en/latest/about.html). This dataset contains 4000 original records, among which : 
* 114 have 1 duplicate record, 
* 141 have 2 duplicate records, 
* 107 have 3 duplicate records, 
* 47 have 4 duplicate records, 
* 19 have 5 duplicate records. 

In total, the dataset contains 1000 duplicate records following a poisson distribution as shown in the figure below. 

<img src="img/image1.png" width="50%"/>

note that the ***duplicate/orginial ratio*** refers to the number of duplcate records against the number of distinct original records. For instance, ***ratio = 5*** correspond to the 19 original records, each of which has ***5*** duplcate records.

#### Fisrt, we install and import the *Python Record Linkage toolkit*

In [1]:
#!pip install recordlinkage
import recordlinkage

#### Then we import the *Febrl2* dataset

In [2]:
from recordlinkage.datasets import load_febrl2
df = load_febrl2()
df

,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id
rec_id,,,,,,,,,,
rec-2778-org,sarah,bruhn,44,forbes street,wintersloe,kellerberrin,4510,vic,19300213,7535316
rec-712-dup-0,jacob,lanyon,5,milne cove,wellwod,beaconsfield upper,2602,vic,19080712,9497788
rec-1321-org,brinley,efthimiou,35,sturdee crescent,tremearne,scarborough,5211,qld,19940319,6814956
rec-3004-org,aleisha,hobson,54,oliver street,inglewood,toowoomba,3175,qld,19290427,5967384
rec-1384-org,ethan,gazzola,49,sheaffe street,bimby vale,port pirie,3088,sa,19631225,3832742
...,...,...,...,...,...,...,...,...,...,...
rec-1487-org,thomas,green,44,tuthill place,holmeleigh,bonny hills,4740,vic,19420210,9334580
rec-1856-org,james,mcneill,42,archibald street,NaN,evans head,2250,nsw,19011207,4837378
rec-3307-org,paige,lock,7,a'beckett street,camboon,carina heights,2290,nsw,19871002,5142242


## 1. Data Profiling - first glance

Since there is no metadata associated to this dataset, we carry out some descriptive analyses by using some basic built-in DataFrame methods.

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5000 entries, rec-2778-org to rec-1143-org
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   given_name     4891 non-null   object
 1   surname        4936 non-null   object
 2   street_number  4777 non-null   object
 3   address_1      4891 non-null   object
 4   address_2      4431 non-null   object
 5   suburb         4950 non-null   object
 6   postcode       5000 non-null   object
 7   state          4952 non-null   object
 8   date_of_birth  4890 non-null   object
 9   soc_sec_id     5000 non-null   object
dtypes: object(10)
memory usage: 429.7+ KB


We can notice that all columns have the non-null condition, but the count of some columns (for instance, give_name, surname) is not 5000. That means the basic *Functional Integrity Constraint*is not fulfilled.

In [4]:
df.describe(include='all')

,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id
count,4891,4936,4777,4891,4431,4950,5000,4952,4890,5000
unique,907,1816,397,2455,2517,1684,1412,19,3785,4089
top,emiily,white,1,forbes street,rosetta village,toowoomba,4701,nsw,19920917,1135598
freq,85,131,142,16,40,46,23,1604,7,6


We can notice that we have 4089 unique ***soc_sec_id*** which corresponds to neither the ***5000*** count nor the ***4000*** original records without duplicates.
That means ***soc_sec_id*** cannot be used to identify in a unique way a person in this dataset.

<font color='red'>***To do***: what can you learn more about the dataset ? </font>

## 2. Standardisation - basic pre-processing techniques

https://recordlinkage.readthedocs.io/en/latest/ref-preprocessing.html

This step is important as standardizing the data into the same format will increase the chances of identifying duplicates.

#### Lowercase --> uppercase

This is the easiest step for text pre-processing which is to standardize your text data set to all â€œlowercaseâ€ or â€œuppercaseâ€. In the example below, we are converting the text in our data set to UPPERCASE.

In [5]:
# a lambda function is a small anonymous function which can only have one expression.
df = df.astype(str).apply(lambda x: x.str.upper())
df

,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id
rec_id,,,,,,,,,,
rec-2778-org,SARAH,BRUHN,44,FORBES STREET,WINTERSLOE,KELLERBERRIN,4510,VIC,19300213,7535316
rec-712-dup-0,JACOB,LANYON,5,MILNE COVE,WELLWOD,BEACONSFIELD UPPER,2602,VIC,19080712,9497788
rec-1321-org,BRINLEY,EFTHIMIOU,35,STURDEE CRESCENT,TREMEARNE,SCARBOROUGH,5211,QLD,19940319,6814956
rec-3004-org,ALEISHA,HOBSON,54,OLIVER STREET,INGLEWOOD,TOOWOOMBA,3175,QLD,19290427,5967384
rec-1384-org,ETHAN,GAZZOLA,49,SHEAFFE STREET,BIMBY VALE,PORT PIRIE,3088,SA,19631225,3832742
...,...,...,...,...,...,...,...,...,...,...
rec-1487-org,THOMAS,GREEN,44,TUTHILL PLACE,HOLMELEIGH,BONNY HILLS,4740,VIC,19420210,9334580
rec-1856-org,JAMES,MCNEILL,42,ARCHIBALD STREET,NAN,EVANS HEAD,2250,NSW,19011207,4837378
rec-3307-org,PAIGE,LOCK,7,A'BECKETT STREET,CAMBOON,CARINA HEIGHTS,2290,NSW,19871002,5142242


#### Stop words removal

Stop words are common words that are removed to provide more importance to more important information in the text. For example :
* in a complete sentence stop words are â€œtheâ€, â€œaâ€, â€œandâ€, etc. 
* for people's name stop words could be â€œMrâ€, â€œMrsâ€, â€œMsâ€, â€œSirâ€, etc. 
* for address,stop words could be â€œStreetâ€, â€œStâ€, â€œPlaceâ€, â€œRdâ€, â€œRoadâ€, etc.
* Un exemple de la liste des mots vides en langue francaise est disponible Ã  l'adresse suivante : https://www.ranks.nl/stopwords/french

For this dataset, there are stop words in the address field â€œaddress_1â€. In the example below, we are removing common stopwords mentioned above.

In [6]:
import nltk
#nltk.download('punkt')
from nltk.tokenize.treebank import TreebankWordDetokenizer

name_stopword = ["STREET", "ST", "PLACE", "RD", "ROAD"] 
# convert an address (a string with several words) into a list of tokens (substring with only one word)
df['address_1_token'] = df['address_1'].apply(nltk.word_tokenize)
# create a new list named "address_1_clean" based on the list named "address_1_token" through list comprehension 
# newlist = [expression for item in iterable if condition == True]
# https://www.w3schools.com/python/python_lists_comprehension.asp
df['address_1_clean'] = df['address_1_token'].apply(lambda x: [word for word in x if word not in name_stopword])
# recreate the full address (as one string) based on the cleaned list of words (tokens)
df['address_1_clean'] = df['address_1_clean'].apply(TreebankWordDetokenizer().detokenize)
df

,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id,address_1_token,address_1_clean
rec_id,,,,,,,,,,,,
rec-2778-org,SARAH,BRUHN,44,FORBES STREET,WINTERSLOE,KELLERBERRIN,4510,VIC,19300213,7535316,"[FORBES, STREET]",FORBES
rec-712-dup-0,JACOB,LANYON,5,MILNE COVE,WELLWOD,BEACONSFIELD UPPER,2602,VIC,19080712,9497788,"[MILNE, COVE]",MILNE COVE
rec-1321-org,BRINLEY,EFTHIMIOU,35,STURDEE CRESCENT,TREMEARNE,SCARBOROUGH,5211,QLD,19940319,6814956,"[STURDEE, CRESCENT]",STURDEE CRESCENT
rec-3004-org,ALEISHA,HOBSON,54,OLIVER STREET,INGLEWOOD,TOOWOOMBA,3175,QLD,19290427,5967384,"[OLIVER, STREET]",OLIVER
rec-1384-org,ETHAN,GAZZOLA,49,SHEAFFE STREET,BIMBY VALE,PORT PIRIE,3088,SA,19631225,3832742,"[SHEAFFE, STREET]",SHEAFFE
...,...,...,...,...,...,...,...,...,...,...,...,...
rec-1487-org,THOMAS,GREEN,44,TUTHILL PLACE,HOLMELEIGH,BONNY HILLS,4740,VIC,19420210,9334580,"[TUTHILL, PLACE]",TUTHILL
rec-1856-org,JAMES,MCNEILL,42,ARCHIBALD STREET,NAN,EVANS HEAD,2250,NSW,19011207,4837378,"[ARCHIBALD, STREET]",ARCHIBALD
rec-3307-org,PAIGE,LOCK,7,A'BECKETT STREET,CAMBOON,CARINA HEIGHTS,2290,NSW,19871002,5142242,"[A'BECKETT, STREET]",A'BECKETT


/!\ We do not systematically remove all above-mentioned stop words in all texts. A careful examination should be carried out before choosing the relevant stop word.

For instance, in France we often use the same name for a street or a square, such as *1 rue jean jaures* or *1 place jean jaures*. In this case, it is necessary to keep the original address without removing any stop word. 

#### Other cleaning

In addition to stop words, we often remove other characters and symbols that are irrelevant to matching. 

For instance, in our running example, the postcode contains "-", "+" and blank spaces (that we can also find in phone numbers). We carry out the following clean-up to keep only numeric values.

In [7]:
# Series.str is a vectorized string functions for Series and Index.
# leading and trailing blank spaces
df['postcode'] = df['postcode'].str.strip()
# find only numeric values in the string 
# [0-9]+ is a regular expression (regex) which means a set of numbers. 
#  - [] indicates a set of something
#  - 0-9 indicates a range of numbers from 0 to 9
#  - + indicates one or several repetitions
# more details are available at: # https://docs.python.org/3/library/re.html
df['postcode']=df['postcode'].str.findall('[0-9]+')
# join all found numeric values together without space
df['postcode'] = df['postcode'].str.join("")
# replace NaN value with ""
df['postcode'] = df['postcode'].fillna("")

We can also remove irrelevant symbols in text. In fact, special symbols will not be helpful in helping to identify similarities in text and should be clean up. 

The below example shows the clean-up done to remove irrelevant symbols in the address field.

In [8]:
df['address_1_clean'] = df['address_1_clean'].str.replace("[\'\".,()*+&\/\-\\\+\!\%:;?]"," ")
df['address_2'] = df['address_2'].str.replace("[\'\".,()*+&\/\-\\\+\!\%:;?]"," ")
df.head()

C:\Users\song\AppData\Local\Temp\ipykernel_18188\1350016437.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['address_1_clean'] = df['address_1_clean'].str.replace("[\'\".,()*+&\/\-\\\+\!\%:;?]"," ")
C:\Users\song\AppData\Local\Temp\ipykernel_18188\1350016437.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['address_2'] = df['address_2'].str.replace("[\'\".,()*+&\/\-\\\+\!\%:;?]"," ")


,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id,address_1_token,address_1_clean
rec_id,,,,,,,,,,,,
rec-2778-org,SARAH,BRUHN,44,FORBES STREET,WINTERSLOE,KELLERBERRIN,4510,VIC,19300213,7535316,"[FORBES, STREET]",FORBES
rec-712-dup-0,JACOB,LANYON,5,MILNE COVE,WELLWOD,BEACONSFIELD UPPER,2602,VIC,19080712,9497788,"[MILNE, COVE]",MILNE COVE
rec-1321-org,BRINLEY,EFTHIMIOU,35,STURDEE CRESCENT,TREMEARNE,SCARBOROUGH,5211,QLD,19940319,6814956,"[STURDEE, CRESCENT]",STURDEE CRESCENT
rec-3004-org,ALEISHA,HOBSON,54,OLIVER STREET,INGLEWOOD,TOOWOOMBA,3175,QLD,19290427,5967384,"[OLIVER, STREET]",OLIVER
rec-1384-org,ETHAN,GAZZOLA,49,SHEAFFE STREET,BIMBY VALE,PORT PIRIE,3088,SA,19631225,3832742,"[SHEAFFE, STREET]",SHEAFFE


<font color='red'>***To do***: write a processing program to clean French phone number, such as +33 600660066, 0033 06 00 66 00 66, 06-00-66-00-66 etc. </font>

## 3. Matching

The matching is carried out in two steps. Fisrt, we pair up records to form candidate links. Then, we apply some matching methodes to determine if each candidate link is a mathch or non match.

### 3.1  Pairing (indexing)

https://recordlinkage.readthedocs.io/en/latest/ref-index.html

After the pre-processing phase, our data set has been pre-processed and considered a clean set of data, we will need to create pairs of records (also known as candidate links). To do so, we use the indexing module. We will study three pairing techniques, namely ***Full Index, Blocking*** and ***Sorted Neighbourhood***. 

#### Record Pairing 1 : Full Index

A Full Index is created based on all the possible combinations of record pairs in the data set. Using Full Index has a risk on data volume as there can be an important number of records pairs. 

In [9]:
from recordlinkage.index import Full

Full_Index_Table = Full().index(df)

For example, based on our data set of 5000 records, a total of 12497500 (5000x5000/2 - 5000) pairs are created using the Full Index.

In [10]:
print(f"Table Records: {len(df)} records, no. of record pairs: {len(Full_Index_Table)} pairs")

Table Records: 5000 records, no. of record pairs: 12497500 pairs


<a id='blocking'></a>
#### Record Pairing 2 : Blocking

Blocking produces record pairs based on the exact same value of one or several *record keys* (for instance, an attribute's values). By blocking based on particular attributes, the number of record pairs can be greatly reduced.

In [11]:
from recordlinkage.index import Block

Block_Index_by_State = Block(left_on="state")
# we can also use a list of columns, such as 
# Block_Index_by_State = Block(left_on=['state', 'address_2'])
Block_Index_by_State_Pairs = Block_Index_by_State.index(df)

For example, by blocking on the attribute â€œStateâ€, only pairs of records from the same state are link with each other and a total of 2768103 pairs are created 

In [12]:
print(f"Table Records: {len(df)} records, no. of record pairs: {len(Block_Index_by_State_Pairs)} pairs")

Table Records: 5000 records, no. of record pairs: 2768103 pairs


Note that having lesser record pairs might not always be the best approach, since there could be a possibility of missing out on actual matches: if there are equivalent records but a typo on the value for the chosen blocking attributes (â€œStateâ€ in our example).

#### Record Pairing 3: Sorted Neighbourhood

*Sorted Neighbourhood* is another alternative that produces pairs with ***nearby*** values. We can pair up records together according to a *record key* (for instance, "Surnameâ€ in our running example).


In [13]:
from recordlinkage.index import SortedNeighbourhood

Neighbour_Index_by_Name = SortedNeighbourhood(left_on="surname", window = 5)
Neighbour_Index_by_Name_Pairs = Neighbour_Index_by_Name.index(df)

A total of 75034 pairs are created using index by Sorted Neighbourhood â€” which is also lesser records compared to *Full Index* & *Block Index*. (It also depends on the value content of the selected column)

In [14]:
print(f"Table Records: {len(df)} records, no. of record pairs: {len(Neighbour_Index_by_Name_Pairs)} pairs")

Table Records: 5000 records, no. of record pairs: 75034 pairs


Note that *Sorted Neighbourhood* is a fuzzy algorithm: it does not require the exact same value of surname: as along as two surnames from two datasets are adjacent after sorting, there will be paired up.

*Sorted Neighbourhood* can also be applied to several columns. In this case, besides using the exact values (as we have done in *Record Pairing 2: Blocking*), we can also build our own *record key* for sorting. 

Here is an example of how a customized *record key* index is built. 
<a id='key_neighour'></a>
<img src="img/image2.png" width="80%"/>


<font color='red'>***To do***: propose an index methode to build your own customized *record key* for the FEBRL2 dataset. Apply the *Sorted Neighbourhood* pairing technique and keep the pairing result in *Neighbour_Index_by_Key_Pairs*  </font>

#### Record Pairing 4: hybrid approach

We can also pair up records with a combination of different approaches. In fact, by using one paring approach, there are chances of missing out on actual matches (in the case of *blocking* and *sorted neigbourhood*) or producing a huge growth in the total number of records (in the case of *full index*). 

We can combine the *blocking* and the *sorted neigbourhood* approaches to reduce the possibility of missing out on actual match records, while still resulting in lesser volume of paired records compared to *Full Index*.

In [15]:
All_Index_Pairs = Block_Index_by_State_Pairs.append(Neighbour_Index_by_Name_Pairs)
#Remove duplicate Pairs 
All_Index_Pairs = All_Index_Pairs.drop_duplicates(keep='first')

In [16]:
print(f"Table Records: {len(df)} records, no. of record pairs: {len(All_Index_Pairs)} pairs")

Table Records: 5000 records, no. of record pairs: 2825974 pairs


### 3.2 Comparison 

https://recordlinkage.readthedocs.io/en/latest/ref-compare.html

Once records are paired up, we apply some matching methodes to determine if there is a match/non match. 

To do so, we first compute similarity of each attribute (string, numeric values, dates or other formats) accroding to a relevant similarity measure. Chose the right similarity measure has a great impact on improving the matching efficiency. It is sometimes necessary to use different similarity measure for different type of attribute values. 

<a id='compare'></a>
For instance, we can compute the similarity score in the following way:  

In [17]:
compare = recordlinkage.Compare()
compare.string('given_name','given_name', method='jarowinkler', threshold=0.85, label = 'given_name_score')
compare.string('surname','surname', method='jarowinkler', threshold=0.85,label = 'surname_score')
#compare.string('street_number','street_number', method='levenshtein', threshold=0.85, label = 'street_number_score')
compare.string('address_1_clean','address_1_clean', method='jarowinkler', threshold=0.85,label = 'address_1_score')
compare.string('address_2','address_2', method='jarowinkler', threshold=0.85, label = 'address_2_score')
compare.string('suburb','suburb', method='jarowinkler', threshold=0.85,label = 'suburb_score')
compare.exact('postcode','postcode', label = 'postcode_score')
compare.exact('state','state', label = 'state_score')
compare.string('date_of_birth','date_of_birth', method='levenshtein', threshold=0.85, label = 'date_of_birth_score')
compare.string('soc_sec_id','soc_sec_id', method='levenshtein',threshold=0.6, label = 'soc_sec_id_score')
comparison_vectors = compare.compute(All_Index_Pairs,df)


<a id='coeff'></a>
<font color='red'>***To do***: Instead of using a threshold, we can also directly compute the absolute similarity score between string (wihtout the parameter *threshold* in compare.string() which returns either 0 or 1). In this case, it is interessing to associate different coefficient values to each attribute's similarity score. Propose a coefficent-based methode to compute attribute similarity </font>

In [18]:
comparison_vectors.head(10)

given_name_score  surname_score  address_1_score  \
rec_id_1      rec_id_2                                                          
rec-712-dup-0 rec-2778-org                0.0            0.0              0.0   
rec-63-dup-0  rec-2778-org                0.0            0.0              0.0   
              rec-712-dup-0               0.0            0.0              0.0   
rec-112-org   rec-2778-org                0.0            0.0              0.0   
              rec-712-dup-0               0.0            0.0              0.0   
              rec-63-dup-0                0.0            0.0              0.0   
rec-2116-org  rec-2778-org                0.0            0.0              0.0   
              rec-712-dup-0               0.0            0.0              0.0   
              rec-63-dup-0                0.0            0.0              0.0   
              rec-112-org                 0.0            0.0              0.0   

                             address_2_score  suburb_score  postcode_score  \
rec_id_1      rec_id_2                                                       
rec-712-dup-0 rec-2778-org               0.0           0.0               0   
rec-63-dup-0  rec-2778-org               0.0           0.0               0   
              rec-712-dup-0              0.0           0.0               0   
rec-112-org   rec-2778-org               0.0           0.0               0   
              rec-712-dup-0              0.0           0.0               0   
              rec-63-dup-0               0.0           0.0               0   
rec-2116-org  rec-2778-org               0.0           0.0               0   
              rec-712-dup-0              0.0           0.0               0   
              rec-63-dup-0               0.0           0.0               0   
              rec-112-org                0.0           0.0               0   

                             state_score  date_of_birth_score  \
rec_id_1      rec_id_2                                          
rec-712-dup-0 rec-2778-org             1                  0.0   
rec-63-dup-0  rec-2778-org             1                  0.0   
              rec-712-dup-0            1                  0.0   
rec-112-org   rec-2778-org             1                  0.0   
              rec-712-dup-0            1                  0.0   
              rec-63-dup-0             1                  0.0   
rec-2116-org  rec-2778-org             1                  0.0   
              rec-712-dup-0            1                  0.0   
              rec-63-dup-0             1                  0.0   
              rec-112-org              1                  0.0   

                             soc_sec_id_score  
rec_id_1      rec_id_2                         
rec-712-dup-0 rec-2778-org                0.0  
rec-63-dup-0  rec-2778-org                0.0  
              rec-712-dup-0               0.0  
rec-112-org   rec-2778-org                0.0  
              rec-712-dup-0               0.0  
              rec-63-dup-0                0.0  
rec-2116-org  rec-2778-org                0.0  
              rec-712-dup-0               0.0  
              rec-63-dup-0                0.0  
              rec-112-org                 0.0

In [19]:
comparison_vectors.describe()

,given_name_score,surname_score,address_1_score,address_2_score,suburb_score,postcode_score,state_score,date_of_birth_score,soc_sec_id_score
count,2.825974e+06,2.825974e+06,2.825974e+06,2.825974e+06,2.825974e+06,2.825974e+06,2.825974e+06,2.825974e+06,2.825974e+06
mean,7.723001e-03,2.126665e-02,2.402358e-03,1.448952e-02,2.708447e-03,1.486213e-03,9.795218e-01,2.007803e-03,8.347564e-04
std,8.754061e-02,1.442719e-01,4.895495e-02,1.194972e-01,5.197222e-02,3.852278e-02,1.416294e-01,4.476352e-02,2.888010e-02
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00
50%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00
75%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00
max,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


The last step is to decide which records belong to the same person. In this example, we keep it simple:

In [20]:
# Sum the comparison results according to record (we calculate the sum of all attribute value of each record)
comparison_vectors.sum(axis=1).value_counts().sort_index(ascending=False)

9.0        518
8.0        676
7.0        449
6.0        184
5.0         62
4.0         19
3.0       1138
2.0      89171
1.0    2721191
0.0      12566
dtype: int64

In [21]:
potential_matches = comparison_vectors[comparison_vectors.sum(axis=1) > 7]
print(len(potential_matches))
potential_matches.head(20)

1194


given_name_score  surname_score  \
rec_id_1       rec_id_2                                          
rec-63-dup-1   rec-63-dup-0                 1.0            1.0   
rec-232-dup-0  rec-232-org                  1.0            1.0   
rec-3123-dup-0 rec-3123-org                 0.0            1.0   
rec-3530-org   rec-3530-dup-2               1.0            1.0   
rec-2924-dup-1 rec-2924-dup-3               1.0            0.0   
rec-1894-dup-1 rec-1894-dup-2               1.0            1.0   
rec-3542-org   rec-3542-dup-1               1.0            1.0   
               rec-3542-dup-2               1.0            1.0   
rec-2119-org   rec-2119-dup-0               1.0            1.0   
rec-2924-org   rec-2924-dup-3               1.0            1.0   
               rec-2924-dup-1               1.0            0.0   
rec-1477-org   rec-1477-dup-1               1.0            1.0   
rec-2119-dup-1 rec-2119-dup-0               1.0            1.0   
               rec-2119-org                 1.0            1.0   
rec-705-dup-2  rec-705-org                  1.0            1.0   
rec-3932-org   rec-3932-dup-0               1.0            1.0   
rec-413-dup-3  rec-413-org                  1.0            1.0   
rec-641-dup-0  rec-641-dup-1                1.0            1.0   
rec-1010-dup-2 rec-1010-org                 1.0            1.0   
rec-2447-org   rec-2447-dup-1               1.0            1.0   

                               address_1_score  address_2_score  suburb_score  \
rec_id_1       rec_id_2                                                         
rec-63-dup-1   rec-63-dup-0                0.0              1.0           1.0   
rec-232-dup-0  rec-232-org                 1.0              1.0           1.0   
rec-3123-dup-0 rec-3123-org                1.0              1.0           1.0   
rec-3530-org   rec-3530-dup-2              1.0              1.0           1.0   
rec-2924-dup-1 rec-2924-dup-3              1.0              1.0           1.0   
rec-1894-dup-1 rec-1894-dup-2              1.0              1.0           1.0   
rec-3542-org   rec-3542-dup-1              1.0              1.0           1.0   
               rec-3542-dup-2              0.0              1.0           1.0   
rec-2119-org   rec-2119-dup-0              1.0              1.0           1.0   
rec-2924-org   rec-2924-dup-3              1.0              1.0           1.0   
               rec-2924-dup-1              1.0              1.0           1.0   
rec-1477-org   rec-1477-dup-1              1.0              1.0           1.0   
rec-2119-dup-1 rec-2119-dup-0              1.0              1.0           1.0   
               rec-2119-org                1.0              1.0           1.0   
rec-705-dup-2  rec-705-org                 1.0              1.0           1.0   
rec-3932-org   rec-3932-dup-0              1.0              0.0           1.0   
rec-413-dup-3  rec-413-org                 1.0              1.0           1.0   
rec-641-dup-0  rec-641-dup-1               1.0              1.0           1.0   
rec-1010-dup-2 rec-1010-org                1.0              1.0           1.0   
rec-2447-org   rec-2447-dup-1              1.0              1.0           1.0   

                               postcode_score  state_score  \
rec_id_1       rec_id_2                                      
rec-63-dup-1   rec-63-dup-0                 1            1   
rec-232-dup-0  rec-232-org                  1            1   
rec-3123-dup-0 rec-3123-org                 1            1   
rec-3530-org   rec-3530-dup-2               1            1   
rec-2924-dup-1 rec-2924-dup-3               1            1   
rec-1894-dup-1 rec-1894-dup-2               0            1   
rec-3542-org   rec-3542-dup-1               1            1   
               rec-3542-dup-2               1            1   
rec-2119-org   rec-2119-dup-0               1            1   
rec-2924-org   rec-2924-dup-3               1            1   
               rec-2924-dup-1               1      

<font color='red'>***To do***: propose other technique to compute similarity by record </font>

The known duplicate record pairs can be found in ***links*** through:

In [22]:
df, links = load_febrl2(return_links=True)

<font color='red'>***To do***: evaluate the efficiency of the matching through the following measures
* precision
* recall
* F1 score = 2*(precision+recall)/(precision+recall)  
</font>

<img src="img/image3.png" width="45%"/>

<font color='red'>***To do***: Improve the matching efficency (for instance, f1 measure) : 
* by trying the <a href='#compare'>comparing algorithms</a> (compare.Numeric(), compare.Date(), even your own comparing algorithm...) 
* by trying other string <a href='#compare'>similarity measures</a> (qgram, smith_waterman...)
* by trying other attribute in <a href='#blocking'>Blocking pairing</a> (with or withot mergint with *Sorted Neighourhood*)  
* by using <a href='#key_neighour'>*Neighbour_Index_by_Key_Pairs*</a> (with or without merging with *Blocking*)
* by using <a href='#coeff'>coeffient-based</a> technique ?
* by using ohter techniques ?    
</font>

## References

* https://towardsdatascience.com/performing-deduplication-with-record-linkage-and-supervised-learning-b01a66cc6882
* https://pbpython.com/record-linking.html
* https://recordlinkage.readthedocs.io/en/latest/about.html
* https://hpi.de/fileadmin/user_upload/fachgebiete/naumann/folien/SS13/DPDC/DPDC_14_SNM.pdf
* https://en.wikipedia.org/wiki/Precision_and_recall